In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)


In [2]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df.head()

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json


In [3]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [4]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("bird", case=False)]
len(data_scoot)
#data_scoot


15

In [7]:
#getting latitudes and longitudes for city as well as coordinates of bird scooters in each cities
coord_length=[]
lat_list=[]
lon_list=[]
all_url=data_scoot["Auto-Discovery URL"]
type(all_url)

for url in all_url:
    
    base_url=url
    #print(base_url)
    data = requests.get(base_url).json()
    #print(data)
    target_url=data["data"]["en"]["feeds"][3]["url"]
    #target_url
    data_coord=requests.get(target_url).json()
    #print(data_coord)
    coordinates=data_coord["data"]["geofencing_zones"][0]["zone_area"]["coordinates"]
    #print(coordinates)
    coord = str(coordinates)[1:-6]
    length=len(coord)
    coord_length.append(length)
    
    Lat=data_coord["data"]["geofencing_zones"][0]["lat"]
    #print(Lat)
    Lon=data_coord["data"]["geofencing_zones"][0]["lon"]
    
    #print(Lon)
    lat_list.append(Lat)
    lon_list.append(Lon)
    


In [8]:


locations = list(zip(lat_list,lon_list))


In [9]:

#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=coord_length,  
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))